In [1]:
import pandas as pd

In [3]:

path_motif_search_file = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\data\helixworks_1280_fast5_2024-02-09_1638\motif_search_barcoded.csv"



squiggle_database_df = pd.read_pickle("squiggle_database.pkl") # All the reads with the squiggles
motif_search_df = pd.read_csv(path_motif_search_file)
motif_search_df.head()

,read_id,ONT_Barcode,HW_Address,orientation,start_end,library_motif
0,0117ec74-7ef1-4e8c-b169-c8ca9a576de4,barcode01,barcode_external01_internal01,+|+|+,75-124|370-419|566-615,ltm8_2x1|ltm8_5x4|ltm8_9x1
1,01575c8a-4a38-4554-9d88-5acbc61059a4,barcode01,barcode_external01_internal01,+|+,176-225|275-324,ltm8_6x1|ltm8_8x7
2,01ecbbe8-550d-433b-a2af-354c33c91dde,barcode01,barcode_external01_internal01,+,108-157,ltm8_1x1
3,062cd1bc-9132-4144-bd56-5aea4e9a5529,barcode01,barcode_external01_internal01,+|+|+|+,154-203|251-300|301-350|351-400,ltm8_2x1|ltm8_4x2|ltm8_5x5|ltm8_6x4
4,06382a54-201e-46cf-beeb-c8d574b30e85,barcode01,barcode_external01_internal01,+|+|+|+|+,149-198|199-248|299-348|349-398|497-546,ltm8_2x1|ltm8_3x4|ltm8_5x5|ltm8_6x8|ltm8_9x3


So what we have here is the motif search database, which associates motifs from the library to a certain part of the seqeunce, not sure whether it maps to pre or post basecalling. Need to figure out how to chop squiggles. Luckily we have all the squiggles we need in the pickle file

In [27]:
def get_squiggle(read_id):
    row = squiggle_database_df.loc[squiggle_database_df['read_id'] == read_id]
    return row['squiggle'].to_numpy()

#print(motif_search_df.iloc[0]['read_id'])
squiggle = get_squiggle(motif_search_df.iloc[0]['read_id'])
print(len(squiggle[0]))

8303


So we can't directly match to chop bits of the fast5 cause we don't know the exact positions. So I'm thinking let's pick the ones with a large number of motifs found and use that as the training set. Gotta find a way to seperate them into bits though


In [35]:

def library_motif_segregation():
    library_motifs = motif_search_df['library_motif'].to_numpy()
    library_motifs_arr = []
    no_motifs = []
    for i in library_motifs:
        motifs_arr = i.split('|')
        library_motifs_arr.append(motifs_arr)
        no_motifs.append(len(motifs_arr))
    return library_motifs_arr, no_motifs


library_motifs_arr, motifs_discovered = library_motif_segregation()
motif_search_df['library_motifs_arr'] = library_motifs_arr
motif_search_df['motifs_discovered'] = motifs_discovered
motif_search_df.head()
    

,read_id,ONT_Barcode,HW_Address,orientation,start_end,library_motif,library_motifs_arr,motifs_discovered
0,0117ec74-7ef1-4e8c-b169-c8ca9a576de4,barcode01,barcode_external01_internal01,+|+|+,75-124|370-419|566-615,ltm8_2x1|ltm8_5x4|ltm8_9x1,"[ltm8_2x1, ltm8_5x4, ltm8_9x1]",3
1,01575c8a-4a38-4554-9d88-5acbc61059a4,barcode01,barcode_external01_internal01,+|+,176-225|275-324,ltm8_6x1|ltm8_8x7,"[ltm8_6x1, ltm8_8x7]",2
2,01ecbbe8-550d-433b-a2af-354c33c91dde,barcode01,barcode_external01_internal01,+,108-157,ltm8_1x1,[ltm8_1x1],1
3,062cd1bc-9132-4144-bd56-5aea4e9a5529,barcode01,barcode_external01_internal01,+|+|+|+,154-203|251-300|301-350|351-400,ltm8_2x1|ltm8_4x2|ltm8_5x5|ltm8_6x4,"[ltm8_2x1, ltm8_4x2, ltm8_5x5, ltm8_6x4]",4
4,06382a54-201e-46cf-beeb-c8d574b30e85,barcode01,barcode_external01_internal01,+|+|+|+|+,149-198|199-248|299-348|349-398|497-546,ltm8_2x1|ltm8_3x4|ltm8_5x5|ltm8_6x8|ltm8_9x3,"[ltm8_2x1, ltm8_3x4, ltm8_5x5, ltm8_6x8, ltm8_...",5


Now let us look at the number of rows with the number of lengths 

In [46]:
for i in sorted(motif_search_df['motifs_discovered'].unique()):

    rows = motif_search_df.loc[motif_search_df['motifs_discovered'] == i]
    print(f"No Motifs = {i} Rows = {rows.shape[0]} ")
    

No Motifs = 1 Rows = 99982 
No Motifs = 2 Rows = 115250 
No Motifs = 3 Rows = 107021 
No Motifs = 4 Rows = 85419 
No Motifs = 5 Rows = 60550 
No Motifs = 6 Rows = 36969 
No Motifs = 7 Rows = 19166 
No Motifs = 8 Rows = 8189 
No Motifs = 9 Rows = 2939 
No Motifs = 10 Rows = 923 
No Motifs = 11 Rows = 342 
No Motifs = 12 Rows = 161 
No Motifs = 13 Rows = 66 
No Motifs = 14 Rows = 30 
No Motifs = 15 Rows = 22 
No Motifs = 16 Rows = 5 
No Motifs = 17 Rows = 2 
No Motifs = 18 Rows = 1 


Let us go for read lengths of 8 first

In [55]:

from tqdm import tqdm

rows = motif_search_df.loc[motif_search_df['motifs_discovered'] == 8]
read_ids = rows['read_id'].to_numpy()
squiggles = []

for read_id in tqdm(read_ids):
    squiggle = get_squiggle(read_id)

    if squiggle is None:
        motif_search_df = motif_search_df[motif_search_df.read_id != read_id]
    else:
        squiggles.append(squiggle)

library_motifs = rows['library_motif'].to_numpy()

print(len(library_motifs))
print(len(squiggles))


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 8189/8189 [03:43<00:00, 36.71it/s]

8189
8189


In [56]:
dataset_df_motif_level = pd.DataFrame()
dataset_df_motif_level['squiggle'] = squiggles
dataset_df_motif_level['motifs'] = library_motifs
dataset_df_motif_level.head()

,squiggle,motifs
0,"[[671, 498, 504, 526, 502, 512, 527, 528, 517,...",ltm8_1x1|ltm8_2x1|ltm8_4x2|ltm8_5x7|ltm8_6x4|l...
1,"[[540, 415, 425, 463, 470, 494, 499, 498, 494,...",ltm8_2x1|ltm8_3x4|ltm8_6x8|ltm8_7x2|ltm8_7x2|l...
2,"[[899, 851, 806, 779, 729, 713, 673, 659, 632,...",ltm8_2x1|ltm8_4x8|ltm8_5x1|ltm8_6x7|ltm8_7x1|l...
3,"[[522, 513, 517, 522, 515, 503, 518, 523, 524,...",ltm8_1x1|ltm8_2x1|ltm8_3x5|ltm8_4x1|ltm8_5x6|l...
4,[],ltm8_2x1|ltm8_4x7|ltm8_5x4|ltm8_6x4|ltm8_6x7|l...
